# Neuro-Visualizer

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader

from NeuroVisualizer.neuro_aux.AEmodel import UniformAutoencoder
from NeuroVisualizer.neuro_aux.utils import get_files

from neuro_viz_helper import get_dataloader_flat, generate_latent_grid, compute_grid_losses

## Train AE Model

In [ ]:
# Adjust this path to your folder
checkpoint_dir = "trainings/models_MLP_mnist"

pt_files = get_files(checkpoint_dir, prefix="model-")
print(f"Found {len(pt_files)} checkpoint files.")

In [ ]:
batch_size = 4 #32

pt_files_subset = pt_files[:]
loader, normalizer = get_dataloader_flat(pt_files_subset, batch_size)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = loader.dataset[0].shape[0]
print(f"Input dimension: {input_dim}")

latent_dim = 2
num_layers = 3
h = [input_dim, 512, 128]  # Aggressive compression
#h = [input_dim, 256, 64]

ae = UniformAutoencoder(input_dim, num_layers, latent_dim, h=h).to(device)

In [ ]:
optimizer = torch.optim.Adam(ae.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

num_epochs = 50

for epoch in range(num_epochs):
    ae.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        recon, _ = ae(batch)
        loss = loss_fn(recon, batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1:02d} - Loss: {avg_loss:.6f}")

In [ ]:
os.makedirs("ae_models", exist_ok=True)
torch.save(ae.state_dict(), "ae_models/ae_model_mlp_mnist.pt")
print("AE saved.")

## Visualize Trajectory

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from NeuroVisualizer.neuro_aux.AEmodel import UniformAutoencoder
from NeuroVisualizer.neuro_aux.utils import get_files, repopulate_model
from NeuroVisualizer.neuro_aux.trajectories_data import get_trajectory_dataloader

In [ ]:
dataset_name = 'mnist'
model_name = 'mlp'
model_folder = 'trainings/models_MLP_mnist'
model_file = 'ae_models/ae_model_mlp_mnist.pt'
batch_size = 4
loss_name = 'test_loss'
whichloss = 'mse'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Get file list
pt_files = get_files(model_folder, prefix="model-")

# Load AE
example_tensor = torch.load(pt_files[0])
input_dim = example_tensor.shape[0]
latent_dim = 2
num_of_layers = 3
h = [input_dim, 512, 128]

best_model = UniformAutoencoder(input_dim, num_of_layers, latent_dim, h=h).to(device)
best_model.load_state_dict(torch.load(model_file))
best_model.eval()

In [ ]:
# ---- Load data ----
from neuro_viz_helper import get_dataloader_flat

trajectory_loader, transform = get_dataloader_flat(pt_files, batch_size, shuffle=False) #[:5] for Subset

In [ ]:
from vision_classification import init_mlp_for_dataset
from neuro_viz_helper import Loss

model = init_mlp_for_dataset(dataset_name, hidden_dims=[254, 64], dropout=0.1).to(device)
loss_obj = Loss(dataset_name, device)

In [ ]:
from neuro_viz_helper import compute_trajectory

trajectory_coordinates, trajectory_models, trajectory_losses = compute_trajectory(
    trajectory_loader,
    best_model,
    transform,
    loss_obj,
    model,
    loss_name,
    whichloss,
    device,
)

In [ ]:
import matplotlib.pyplot as plt

# Convert tensor to CPU & numpy
print(trajectory_losses)
loss_values = trajectory_losses.cpu().numpy()

plt.figure(figsize=(8, 4))
plt.plot(range(len(loss_values)), loss_values, marker='o', color='blue')
plt.title('Trajectory Losses over Training Steps')
plt.xlabel('Checkpoint Index')
plt.ylabel('NLL Loss (log scale)')
plt.grid(True)
plt.show()

In [ ]:
# 1️⃣ Generate grid in latent space
from neuro_viz_helper import generate_latent_grid, compute_grid_losses
xx, yy, grid_coords = generate_latent_grid(
    min_map=-1, max_map=1, xnum=3, device=device
)

# 2️⃣ Decode grid and compute losses
model = init_mlp_for_dataset(dataset_name, hidden_dims=[254, 64], dropout=0.1).to(device)

grid_losses = compute_grid_losses(
    grid_coords,
    transform,
    best_model,
    model,
    loss_obj,
    loss_name,
    whichloss,
    device
)

# 3️⃣ Reshape to grid
grid_losses = grid_losses.view(xx.shape)

In [ ]:
print(grid_losses.min().item(), grid_losses.max().item())

In [ ]:
grid_losses

In [ ]:
rec_grid_models = best_model.decoder(grid_coords)
rec_grid_models = rec_grid_models*transform.std.to(device) + transform.mean.to(device)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker
import numpy as np

# === PREPARE LOSSES (flip sign since NLL is negative) ===
grid_losses_pos = -grid_losses.detach().cpu().numpy()
traj_losses_pos = -trajectory_losses.detach().cpu().numpy()

# Avoid zeros or negative values for LogNorm
grid_losses_pos[grid_losses_pos <= 1e-3] = 1e-3
traj_losses_pos[traj_losses_pos <= 1e-3] = 1e-3

# === SHARED COLOR SCALE ===
all_losses = np.concatenate([grid_losses_pos.flatten(), traj_losses_pos])
vmin = np.clip(all_losses.min() / 1.2, 1e-3, None)
vmax = all_losses.max() * 1.2

if vmin >= vmax or np.isclose(vmin, vmax):
    vmax = vmin * 10
    print(f"Adjusted nearly-constant losses: vmin={vmin}, vmax={vmax}")

levels = np.logspace(np.log10(vmin), np.log10(vmax), 30)
shared_norm = LogNorm(vmin=vmin, vmax=vmax)
shared_cmap = 'viridis' #hot

# === BEGIN PLOTTING ===
fig, ax = plt.subplots(figsize=(8, 6))

# -- 1️⃣ Loss Landscape Contour --
contour = ax.contourf(
    xx.cpu().numpy(),
    yy.cpu().numpy(),
    grid_losses_pos,
    levels=levels,
    norm=shared_norm,
    cmap=shared_cmap
)
cbar = plt.colorbar(contour, ax=ax, shrink=0.8)
cbar.ax.set_ylabel('-NLL Loss (Higher is Better)', fontsize=12)

# -- 2️⃣ Trajectory Lines --
z = trajectory_coordinates.cpu().numpy()
for i in range(len(z) - 1):
    ax.plot([z[i, 0], z[i + 1, 0]], [z[i, 1], z[i + 1, 1]], color='k', linewidth=1)

# -- 3️⃣ Trajectory Points with SAME Color Mapping --
sc = ax.scatter(
    z[:, 0], z[:, 1],
    c=traj_losses_pos,
    cmap=shared_cmap,
    norm=shared_norm,
    s=40,
    edgecolors='k',
    #label='Trajectory Points'
)

# -- 4️⃣ OPTIONAL: Density Contours --
try:
    from NeuroVisualizer.neuro_aux.utils import get_density
    density = get_density(rec_grid_models.detach().cpu().numpy(), type='inverse', p=2)
    density = density.reshape(xx.shape)
    density_levels = np.logspace(
        np.log10(max(density.min(), 1e-3)),
        np.log10(density.max()),
        15
    )
    CS_density = ax.contour(
        xx.cpu().numpy(), yy.cpu().numpy(), density,
        levels=density_levels,
        colors='white',
        linewidths=0.8
    )
    ax.clabel(CS_density, fmt=ticker.FormatStrFormatter('%.1f'), fontsize=7)
except Exception as e:
    print("Density contour skipped:", e)

# -- 5️⃣ Labels, Grid, Style --
ax.set_title('Loss Landscape with Training Trajectory', fontsize=14)
ax.set_xlabel('Latent Dimension 1', fontsize=12)
ax.set_ylabel('Latent Dimension 2', fontsize=12)
#ax.legend(loc='best')
ax.grid(True, linestyle='--', alpha=0.3)

# -- 6️⃣ Show or Save --
plt.show()

In [ ]:
# ✅ Save to PDF
os.makedirs('plots', exist_ok=True)
fig.savefig('plots/loss_landscape_with_trajectory.pdf', dpi=300, bbox_inches='tight', format='pdf')
print("Saved PDF to plots/loss_landscape_with_trajectory.pdf")

plt.show()